# HMM Views IC Frontier: Oldest Country ETFs

## 1. Data Fetching

### 1.1 Model configuration

In [1]:
import os
import sys
import datetime as dt
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from hmmlearn import hmm
import cvxportfolio as cp
import alphamodel as am

config = {'name': 'bl_hmm_sim',
          'universe':
              {'list': ['SPY', 'EWA', 'EWC', 'EWG', 'EWH', 'EWJ', 'EWS', 'EWU', 'EWW'],
               'ticker_col': 'Symbol',
               'risk_free_symbol': 'USDOLLAR'},
          'data':
              {'name': 'eod_returns',
               'source': 'quandl',
               'table': 'EOD',
               'api_key': "6XyApK2BBj_MraQg2TMD"},
          'model':
              {'start_date': '19970102',
               'end_date': '20011231',
               'halflife': 20,
               'min_periods': 3,
               'hidden_states': 2,
               'train_len': 750,
               'data_dir': '/Users/razvan/PyRepo/research_masc/data_store/sp100_ss/',
               'returns':
                   {'sampling_freq': 'daily'},
               'covariance':
                    {'method' : 'FF5',
                     'sampling_freq' : 'monthly',
                     'train_days': 90}
              }
         }

/Users/razvan/miniconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


### 1.2 Fetch return data

In [2]:
# Fetch returns / volumes
ss = am.SingleStockHMM(config)
ss.train(force=True)

downloading SPY from 19970102 to 20011231
downloading EWA from 19970102 to 20011231
downloading EWC from 19970102 to 20011231
downloading EWG from 19970102 to 20011231
downloading EWH from 19970102 to 20011231
downloading EWJ from 19970102 to 20011231
downloading EWS from 19970102 to 20011231
downloading EWU from 19970102 to 20011231
downloading EWW from 19970102 to 20011231
downloading USDOLLAR from 19970102 to 20011231
Removing these days from dataset:
            nan price  nan open price  nan close price  nan volumes
1999-04-02          9             NaN              NaN          NaN
2001-09-13          9             NaN              NaN          NaN
2001-09-14          9             NaN              NaN          NaN
          remaining nan price  remaining nan open price  \
SPY                         0                         0   
EWA                         0                         0   
EWC                         0                         0   
EWG                         0    

True

## 2. HMM information coefficient frontier

Hyperparameters:
- hmm_mode: return prediction, 'e' (expectation), 't' (regime over probability threshold)
- preprocess: raw data or exponential decay
- train_len: length of training data
- halflife: halflife of exponential decay

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

ic_vs_params = {}

for hmm_mode in ['e', 't']:
    for preprocess in [None, 'exponential']:
        for train_len in range(50, 1501, 50):
            for halflife in [65, 130, 250, 500, 750]:
                # Avoid cases we don't want to run
                if (not preprocess and halflife > 65) or train_len < halflife:
                    continue
                
                # Set the parameters for the prediction
                ss.cfg['train_len'] = train_len
                ss.cfg['halflife'] = halflife

                # Predict and gather metrics
                ss.predict(mode=hmm_mode, preprocess=preprocess)
                ic = ss.prediction_quality(statistic='information_coefficient', print=False)

                # Save down metrics together with parameters
                key = hmm_mode + preprocess + str(train_len) + str(halflife)
                ic_vs_params[key] = [hmm_mode, preprocess, train_len, ic.loc[5, 'mean'], ic.loc[5, 'std']]

# Save down values in .csv
ic_df = pd.DataFrame.from_dict(ic_vs_params, orient='index')
ic_df.columns = ['hmm_mode', 'preprocess', 'train_len', 'ic_5d_mean', 'ic_5d_std']
ic_df.to_csv(ss.cfg['data_dir'] + 'hmm_ic.csv', index=False)

Running for 1997 May 31
Running for 1997 Jun 30
Running for 1997 Jul 31
Running for 1997 Aug 31
Running for 1997 Sep 30
Running for 1997 Oct 31
Running for 1997 Nov 30
Running for 1997 Dec 31
Running for 1998 Jan 31
Running for 1998 Feb 28
Running for 1998 Mar 31
Running for 1998 Apr 30
Running for 1998 May 31
Running for 1998 Jun 30
Running for 1998 Jul 31
Running for 1998 Aug 31
Running for 1998 Sep 30
Running for 1998 Oct 31
Running for 1998 Nov 30
Running for 1998 Dec 31
Running for 1999 Jan 31
Running for 1999 Feb 28
Running for 1999 Mar 31
Running for 1999 Apr 30
Running for 1999 May 31
Running for 1999 Jun 30
Running for 1999 Jul 31
Running for 1999 Aug 31
Running for 1999 Sep 30
Running for 1999 Oct 31
Running for 1999 Nov 30
Running for 1999 Dec 31
Running for 2000 Jan 31
Running for 2000 Feb 29
Running for 2000 Mar 31
Running for 2000 Apr 30
Running for 2000 May 31
Running for 2000 Jun 30
Running for 2000 Jul 31
Running for 2000 Aug 31
Running for 2000 Sep 30
Running for 2000

/Users/razvan/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/razvan/PyRepo/alphamodel/alphamodel/ss_hmm.py:241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idyos[day].loc[idyos[day] < 0] = 0
/Users/razvan/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s